In [95]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline

In [96]:
def read_merge(otu, alpha_div, mapping, alpha_metric="Shannon.effective", mapping_classes="Phenotype"):
    otu_table = pd.read_table(otu, header=0, index_col=0).T
    mapping_file = pd.read_table(mapping, header=0, index_col=0)
    alpha_diversity = pd.read_table(alpha_div, header=0, index_col=0)
    merged = pd.concat([alpha_diversity[alpha_metric], otu_table, mapping_file[mapping_classes]], axis=1 )
    return merged

In [97]:
cc_merged = read_merge("CC/OTUs_Table-norm-rel.tab", "CC/alpha-diversity.tab", "CC/mapping_file.tab")
tissue_merged = read_merge("tissue/OTUs_Table-norm-rel.tab", "tissue/alpha-diversity.tab", "tissue/mapping_file.tab")

In [98]:
X_cc = cc_merged.drop("Phenotype", axis=1)
y_cc = cc_merged.Phenotype
X_tissue = tissue_merged.drop("Phenotype", axis=1)
y_tissue = tissue_merged.Phenotype

In [99]:
X_cc_train, X_cc_test, y_cc_train, y_cc_test = train_test_split(X_cc, y_cc, random_state=42)
X_tissue_train, X_tissue_test, y_tissue_train, y_tissue_test = train_test_split(X_tissue, y_tissue, random_state=42)

In [93]:
clf = RandomForestClassifier(n_estimators=500,random_state=42,criterion='entropy', n_jobs=-1)

cc_RF = clf.fit(X_cc_train, y_cc_train)
tissue_RF = clf.fit(X_tissue_train, y_tissue_train)

print ("Accuracy of Random Forest Classifier: "+str(clf_cc.score(X_cc_test,y_cc_test)))

Accuracy of Random Forest Classifier: 0.785714285714
